# 1) Objective

A new alert with the most popular hashtags/URLs from business' followers in a recent timeframe

# 2) Current statistics on Production (real data)

### General

* 87 active businesses
* 157 handles from active businesses
* **118 Twitter** handles from active businesses
* **39 Instagram** handles from active businesses

# 3) Twitter

## 3.1) Analyzing the distribution of followers count for Twitter

* 24809421 total Twitter followers from active businesses

* 210249 average Twitter followers from active businesses

* 2582 1st quartile Twitter followers from active businesses (25% have 2582 or less followers)
* 12264 2nd quartile Twitter followers from active businesses (50% have 12264 or less followers)
* 70974 3rd quartile Twitter followers from active businesses (75% have 70974 or less followers)
* 12741830 maximum Twitter followers from active businesses

## 3.2) Required requests for Twitter API

    - https://dev.twitter.com/rest/public
    - https://dev.twitter.com/rest/public/rate-limits

* To obtain followers' ids. Results are given in groups of 5,000 user IDs.
    * **Reference:** https://dev.twitter.com/rest/reference/get/followers/ids
    * **Rate limit:** 15 requests / 15-min window (user auth or app auth)
    * GET http://api.twitter.com/1.1/followers/ids.json?screen_name={business_handle}
* To obtain 3.200 more recent tweets from each follower
    * **Reference:** https://dev.twitter.com/rest/reference/get/statuses/user_timeline
    * **Rate limit:** 180 requests (user auth) or 300 requests (app auth) / 15-min window
    * GET http://api.twitter.com/1.1/statuses/user_timeline.json?user_id={follower_ID}
* To create lists
    * **Reference:** https://dev.twitter.com/rest/reference/post/lists/subscribers/create
    * POST https://api.twitter.com/1.1/lists/subscribers/create.json
* To read tweets from lists:
    * **Reference:** https://dev.twitter.com/rest/reference/get/lists/statuses
    * **Rate limit:** 180 requests / 15-min window (user auth or app auth)
    * GET https://api.twitter.com/1.1/lists/statuses.json

## 3.3) Strategy 

### Loading followers on Twitter 

* The application already loads the followers ids from actors, but it is still dependent on the existence of configuration for InfluentialFollower. A first step would be to turn these two mecanisms independent from each other, since the followers will be also needed for the new feature of "what followers are talking about"
* Another mecanism must be created to load the followers since the oldest ones and store all followers of our active businesses using the Twitter API as described previously.
* With all followers in our database, the next step is to find a statistically significant sample of followers for each actor of each active business and store it in a new Twitter list (which may be associated to our account or the account of the business). The maximum number of lists is 1000 which is a much greater limit above the 118 Twitter handles of our current active businesses. In order to prevent this limit of being exceeded in the future, it could be reasonable to creat the lists on clients' accounts.

### Loading tweets from followers through Rules (*primary alternative*)

* Create mechanism for allowing the creation of rules for "followers", so that not only the current logic of Trending Hashtags/URLs can be applied over it but all other existent alerts. 

### Manually loading tweets from followers (*secondary alternative*)

* With the lists configured, read tweets from each list using the API as previously described. Twitter API's rate limit for this operation allows us to load all lists of all actors (118) of all current active businesses.

#### Approach for scoring popularity for manually loaded tweets

* **First:** Simple TF (Term-Frequency) calculation for Hashtags and URLs.
* **Second:** Based on the obtained data and quantities, define a threshold T such that if TF/n_followers > T then the Hashtag/URL is considered *what followers are talking about*. Consider the attribute "created_at" in order to discard old "popular" hashtags/URLs. Define a timeframe threshold too.
* **Third:** Both thresholds of **second** topic can be defined with a default value a configurable through a new FollowersTalkingAboutConfiguration on Admin.


# 4) Instagram

*AT LEAST FOR NOW, INSTAGRAM WILL NOT BE CONSIDERED. ALTHOUGH THE ANALYSIS BELOW PROVIDES AN OVERVIEW OF ITS API AND STATISTICS*

## 4.1) Analyzing the distribution of followers count for Instagram

* 1768969 total Instagram followers from active businesses

* 60998 average Instagram followers from active businesses

* 10027 1st quartile Instagram followers from active businesses
* 31549 2nd quartile Instagram followers from active businesses
* 109050 3rd quartile Instagram followers from active businesses
* 424938 maximum Instagram followers from active businesses

## 4.2) Required requests for Instagram API

    - https://instagram.com/developer
    - https://instagram.com/developer/limits

* To obtain followers' ids
    * **Reference:** https://instagram.com/developer/endpoints/relationships/#get_users_followed_by
    * **Rate limit:** 60/hour
    * GET https://api.instagram.com/v1/users/{user-id}/followed-by?access_token=ACCESS-TOKEN
* To obtain more recent activities from each follower
    * **Reference:** https://instagram.com/developer/endpoints/users/#get_users_media_recent
    * **Rate limit:** 100/hour
    * GET https://api.instagram.com/v1/users/{user-id}/media/recent/?access_token=ACCESS-TOKEN
* To obtain more recent likes from each follower
    * **Reference:** https://instagram.com/developer/endpoints/users/#get_users_feed_liked
    * **Rate limit:** 100/hour
    * GET https://api.instagram.com/v1/users/self/media/liked?access_token=ACCESS-TOKEN